In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

import qubx
%qubxd

%load_ext autoreload
%autoreload 2
from qubx import lookup
from qubx.core.basics import Instrument, FuturesInfo
from qubx.utils.marketdata.binance import get_binance_symbol_info_for_type
from qubx.utils.pandas import scols

 >  [dev] installed cython rebuilding hook
2024-02-29 15:25:10.012 [ ℹ️ ] Loaded 284 for binance.um
2024-02-29 15:25:10.091 [ ℹ️ ] Loaded 61 for binance.cm
2024-02-29 15:25:10.129 [ ℹ️ ] Loaded 2495 for binance
2024-02-29 15:25:10.558 [ ℹ️ ] Updates 7 for DUKASCOPY

                   ╻
   ┏┓      ╻       ┃  ┏┓       Quantitative Backtesting Environment 
   ┃┃  ┓┏  ┣┓  ┏┓  ┃  ┏┛       
   ┗┻  ┗┻  ┗┛  ┗   ┃  ┗━       (c) 2024,  ver. Dev
                   ╹       



# mstruct -> Struct

In [2]:
from qubx.utils import Struct

In [3]:
y = Struct(Struct(x=1), b=Struct(x=1), c = 555)
y

Struct(x=1, b=Struct(x=1), c=555)

In [4]:
y | {'a1': 2}

Struct(x=1, b=Struct(x=1), c=555, a1=2)

In [5]:
y.ast = 1

In [6]:
dict(s=1)

{'s': 1}

In [7]:
y

Struct(x=1, b=Struct(x=1), c=555, a1=2, ast=1)

# Market info and symbols

In [8]:
s0 = lookup['BINANCE:ETH.*']
s1 = lookup['DUKAS:EURGBP']

In [9]:
(
    lookup.find_aux_instrument_for(s0[0], 'USDT'), 
    lookup.find_aux_instrument_for(s0[1], 'USDT'), 
    lookup.find_aux_instrument_for(s1[0], 'USD'),
)

(Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE', base='BTC', quote='USDT', margin_symbol=None, min_tick=0.01, min_size_step=1e-05, min_size=1e-05, futures_info=None, _aux_instrument=None),
 None,
 Instrument(symbol='GBPUSD', market_type='FX', exchange='DUKAS', base='GBP', quote='USD', margin_symbol='USD', min_tick=1e-05, min_size_step=1, min_size=1000, futures_info=None, _aux_instrument=None))

: 